# Text engine for rhymes generator

In [77]:
import re
import json
from typing import Callable

## Load vocabulary from internet and prepare for usage

In [2]:
def upper_repl(match):
        return match.group(1).upper()
    
def upperAccents(text):   
    removed_dash = re.sub(r'-', '', text)
    return re.sub(r'\^(\w)', upper_repl, text)

In [3]:
with open('all_accents.tsv', 'r', encoding='utf-8') as file:
    transformed_text = upperAccents(file.read())
    raw_vocabulary = transformed_text.split('\n')
# raw_vocabulary = {word: accent for word, accent in [raw_line.split('\t') for raw_line in raw_vocabulary]}

In [4]:
accents = {}
for pair in raw_vocabulary:
    if len(words := pair.split('\t')) == 2:
        accents[words[0]] = words[1]

In [79]:
#save
with open('accents.json', 'w', encoding='utf-8') as file:
    accents = json.dump(accents, file)

#load
with open('accents.json', 'r', encoding='utf-8') as file:
    accents = json.load(file)

## Apply acccents on text

In [52]:
def tokenizer(text: str):
    cleaned = re.sub(r'[.,:;!?`\^]', '', text.lower())
    cleaned_dash = re.sub(r'-', ' ', cleaned)
    replace_n = re.sub(r'\n', ' $ ', cleaned_dash)
    return replace_n.split()

def apply_accents(text: str, accents: dict, tokenizer: Callable) -> str:
    words = tokenizer(text)
    accented_words = []
    for word in words:
        word = re.sub(r'ё', 'Ё', word)
        accented_word = accents.get(word, word)
        accented_words.append(accented_word)
    
    return ' '.join(accented_words)
    

In [55]:
text = '''оставить след в душе оксаны
олег хотел но попадал
то в след аркадия глубокий
то в глеба юркие следы
'''

In [56]:
accented_text = apply_accents(text, accents, tokenizer)
accented_text

'остАвить слЕд в душЕ оксАны $ Олег хотЕл нО попадАл $ тО в слЕд аркАдия глубОкий $ тО в глЕба Юркие следЫ $'

## Turn words into rythmical pattern

In [61]:
def replaceVowels(text):
    accent_symbol = 'V'
    nonaccent_symbol = '-'
    vowels = 'ёуеэоаыяию'
    only_vowels = re.sub(rf'[^{vowels}{vowels.upper()}$]', '', text)
    change_accents = re.sub(rf'[{vowels.upper()}]', accent_symbol, only_vowels)
    change_nonaccents = re.sub(rf'[{vowels}]', nonaccent_symbol, change_accents)
    
    return change_nonaccents
    

def getRythmicalPattern(text, accents, tokenizer):
    accented_text = apply_accents(text, accents, tokenizer)
    pattern = replaceVowels(accented_text)
    return pattern
    

In [74]:
text = '''Идёт направо — песнь заводит,
Налево — сказку говорит.
Там чудеса: там леший бродит,
Русалка на ветвях сидит;
Там на неведомых дорожках
Следы невиданных зверей;
Избушка там на курьих ножках
Стоит без окон, без дверей;
'''

In [75]:
apply_accents(text, accents, tokenizer)

'идЁт напрАво — пЕснь завОдит $ налЕво — скАзку говорИт $ тАм чудесА тАм лЕший брОдит $ русАлка нА ветвЯх сидИт $ тАм нА невЕдомых дорОжках $ следЫ невИданных зверЕй $ избУшка тАм нА кУрьих нОжках $ стоИт бЕз Окон бЕз дверЕй $'

In [76]:
for line in getRythmicalPattern(text, accents, tokenizer).split('$'):
    print(line)

-V-V-V-V-
-V-V---V
V--VVV-V-
-V-V-V-V
VV-V---V-
-V-V---V
-V-VVV-V-
-VVV-V-V

